In [310]:
from pyspark.sql import SparkSession
spark_hdfs = SparkSession.builder.appName("Challenge Spark").getOrCreate()


## Challenge Report Nguon Von - Su Dung Von - Daily

### Load 2 file từ hadoop lên

In [268]:
fact_ln_customer = spark_hdfs.read.csv("hdfs://172.30.2.147:9000/demo_directory/challenge/fact_ln_customer.csv", 
                           header = "true", inferSchema = "true")

In [269]:
fact_dp_customer = spark_hdfs.read.csv("hdfs://172.30.2.147:9000/demo_directory/challenge/fact_dp_customer.csv", 
                           header = "true", inferSchema = "true")

In [270]:
fact_ln_customer.show(3)

+----------+----------+---------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+
|process_dt|account_id|   account_type|customer_id|customer_type|interest_rate|cof_rate|    balance| open_date|maturity_date|close_date|
+----------+----------+---------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+
|2023-01-01|         6|  CV VND LS USD|          6|     Business|        15.09|   17.52|34529000000|2022-03-18|   2023-08-28|2023-08-28|
|2023-01-01|        23|CV THONG THUONG|         23|     Business|        12.11|    14.4|11704000000|2022-06-26|   2023-11-09|2023-11-09|
|2023-01-01|         1|CV THONG THUONG|          1|     Business|        10.27|   11.53|15523000000|2022-03-19|   2024-01-21|      NULL|
+----------+----------+---------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+
only showing top 3 rows



In [271]:
fact_dp_customer.show(5)

+----------+----------+------------+-----------+-------------+-------------+--------+----------+----------+-------------+----------+
|process_dt|account_id|account_type|customer_id|customer_type|interest_rate|vof_rate|   balance| open_date|maturity_date|close_date|
+----------+----------+------------+-----------+-------------+-------------+--------+----------+----------+-------------+----------+
|2023-01-01|       100|  Individual|        100|          VIP|         5.21|     7.4|2393700000|2023-01-01|   2023-04-19|2023-04-19|
|2023-01-01|       555|  Individual|        555|      Regular|         7.73|    9.08|4623900000|2023-01-01|   2023-12-27|2023-12-28|
|2023-01-01|      1075|  Individual|       1075|      Regular|         4.64|    6.91|4620100000|2023-01-01|   2023-04-14|2023-04-14|
|2023-01-01|      1980|  Individual|       1980|      Regular|         7.73|    9.09|3811200000|2023-01-01|   2023-02-20|2023-02-20|
|2023-01-01|      3499|  Individual|       3499|      Regular|       

### Tạo dim_funding_structure

In [22]:
dim_funding_structure = spark_hdfs.createDataFrame(
    [
        ("DP", "A. Nguôn vốn", -1, 0, 1000000),
        ("LN", "B. Sử dụng vốn", -1, 0, 2000000),
        ("DP01", "I/ Tiền gửi TCKT & DC: Theo kỳ hạn", 1, 1, 1010000),
        ("DP02", "II/ Tiền gửi TCKT & DC: Theo loại hình", 1, 1, 1020000),
        ("LN01", "I/ Dư nợ TCKT & DC: Theo kỳ hạn", 2, 1, 2010000),
        ("LN02", "II/ Dư nợ TCKT & DC: Theo loại hình", 2, 1, 2020000),
        ("DP01001", "a. KKH", 3, 2, 1010100),
        ("DP01002", "b. 1-3W", 3, 2, 1010101),
        ("DP01003", "c. 1-5 tháng", 3, 2, 1010102),
        ("DP01004", "d. 6 tháng", 3, 2, 1010103),
        ("DP01005", "e. 7 tháng", 3, 2, 1010104),
        ("DP01006", "f. 8-11 tháng", 3, 2, 1010105),
        ("DP01007", "g. 12 tháng", 3, 2, 1010106),
        ("DP01008", "h. 13 tháng", 3, 2, 1010107),
        ("DP01009", "i. 14-23 tháng", 3, 2, 1010108),
        ("DP010010", "j. 24 đến 36 tháng", 3, 2, 1010109),
        ("DP02001", "a. Doanh nghiệp", 4, 2, 1020100),
        ("DP02002", "b. Cá nhân", 4, 2, 1020101),
        ("LN01001", "a. Nợ QH", 5, 2, 2010100),
        ("LN01002", "b. CV THONG THUONG", 5, 2, 2010101),
        ("LN01003", "c. CV CCSTK", 5, 2, 2010102),
        ("LN01004", "d. CV VNĐ lãi suất USD", 5, 2, 2010103),
        ("LN01005", "e. CV ngắn hạn", 5, 2, 2010104),
        ("LN01006", "f. CV trung hạn", 5, 2, 2010105),
        ("LN01007", "g. CV dài hạn <= 120T", 5, 2, 2010106),
        ("LN01008", "h. CV dài hạn <= 180T", 5, 2, 2010107),
        ("LN01009", "i. CV dài hạn > 180T", 5, 2, 2010108),
        ("LN010010", "j. CV ngắn hạn, HSRR ≥ 120%", 5, 2, 2010109),
        ("LN010011", "k. CV trung hạn, HSRR ≥ 120%", 5, 2, 2010110),
        ("LN010012", "l. CV dài hạn ≤ 120T, HSRR ≥ 120%", 5, 2, 2010111),
        ("LN010013", "m. CV dài hạn ≤ 180T, HSRR ≥ 120%", 5, 2, 2010112),
        ("LN010014", "n. CV dài hạn > 180T, HSRR ≥ 120%", 5, 2, 2010113),
        ("LN02002", "a. Doanh nghiệp", 6, 2, 2020100),
        ("LN02003", "b. Cá nhân", 6, 2, 2020101)
    ], ["funding_code", "funding_name", "funding_parent_id", "funding_level", "sortorder"]
)

In [28]:
## cheat tạo auto increasment

from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = Window.orderBy(F.lit(1))
dim_funding_structure = dim_funding_structure.withColumn("funding_id", 
                                                         F.row_number().over(window_spec))

dim_funding_structure = dim_funding_structure.select("funding_id", "funding_code", 
                                                     "funding_name", "funding_parent_id", 
                                                     "funding_level", "sortorder")

In [31]:
dim_funding_structure.show(35)

+----------+------------+--------------------+-----------------+-------------+---------+
|funding_id|funding_code|        funding_name|funding_parent_id|funding_level|sortorder|
+----------+------------+--------------------+-----------------+-------------+---------+
|         1|          DP|        A. Nguôn vốn|               -1|            0|  1000000|
|         2|          LN|      B. Sử dụng vốn|               -1|            0|  2000000|
|         3|        DP01|I/ Tiền gửi TCKT ...|                1|            1|  1010000|
|         4|        DP02|II/ Tiền gửi TCKT...|                1|            1|  1020000|
|         5|        LN01|I/ Dư nợ TCKT & D...|                2|            1|  2010000|
|         6|        LN02|II/ Dư nợ TCKT & ...|                2|            1|  2020000|
|         7|     DP01001|              a. KKH|                3|            2|  1010100|
|         8|     DP01002|             b. 1-3W|                3|            2|  1010101|
|         9|     DP01

### Cập nhập các loại hình

In [272]:
from pyspark.sql.functions import col, when, datediff, floor

fact_ln_customer = fact_ln_customer.withColumn(
    "funding_id_ky_han",
    when(datediff(col("maturity_date"), col("open_date")) == 0, 'DP01001')  # KKH
    .when(datediff(col("maturity_date"), col("open_date")).between(7, 21), 'DP01002')  # 1-3W
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30).between(1, 5), 'DP01003')  # 1-5 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30) == 6, 'DP01004')  # 6 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30) == 7, 'DP01005')  # 7 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30).between(8, 11), 'DP01006')  # 8-11 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30) == 12, 'DP01007')  # 12 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30) == 13, 'DP01008')  # 13 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30).between(14, 23), 'DP01009')  # 14-23 tháng
    .when(floor(datediff(col("maturity_date"), col("open_date")) / 30).between(24, 36), 'DP010010')  # 24-36 tháng
    .otherwise(None)
)

fact_ln_customer = fact_ln_customer.withColumn(
    "funding_id_loai_hinh",
    when(col("account_type") == 'Business', 'DP02001')
    .when(col("account_type") == 'Individual', 'DP02002')
    .otherwise(None)
)

In [273]:
fact_ln_customer.show()

+----------+----------+---------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+-----------------+--------------------+
|process_dt|account_id|   account_type|customer_id|customer_type|interest_rate|cof_rate|    balance| open_date|maturity_date|close_date|funding_id_ky_han|funding_id_loai_hinh|
+----------+----------+---------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+-----------------+--------------------+
|2023-01-01|         6|  CV VND LS USD|          6|     Business|        15.09|   17.52|34529000000|2022-03-18|   2023-08-28|2023-08-28|          DP01009|                NULL|
|2023-01-01|        23|CV THONG THUONG|         23|     Business|        12.11|    14.4|11704000000|2022-06-26|   2023-11-09|2023-11-09|          DP01009|                NULL|
|2023-01-01|         1|CV THONG THUONG|          1|     Business|        10.27|   11.53|15523000000|2022-03-19|   2024-0

In [274]:
from pyspark.sql.functions import col, when, datediff, floor

fact_dp_customer = fact_dp_customer.withColumn(
    "funding_id_ky_han",
    when(
        (floor(datediff(col("maturity_date"), col("open_date")) / 365) <= 1) & 
        (col("vof_rate") / col("interest_rate") >= 1.2), 'LN010010'  # CV ngắn hạn, HSRR ≥ 120%
    )
    .when(
        (floor(datediff(col("maturity_date"), col("open_date")) / 365).between(1, 5)) & 
        (col("vof_rate") / col("interest_rate") >= 1.2), 'LN010011'  # CV trung hạn, HSRR ≥ 120%
    )
    .when(
        (floor(datediff(col("maturity_date"), col("open_date")) / 365).between(5, 10)) & 
        (col("vof_rate") / col("interest_rate") >= 1.2), 'LN010012'  # CV dài hạn ≤ 120T, HSRR ≥ 120%
    )
    .when(
        (floor(datediff(col("maturity_date"), col("open_date")) / 365).between(10, 15)) & 
        (col("vof_rate") / col("interest_rate") >= 1.2), 'LN010013'  # CV dài hạn ≤ 180T, HSRR ≥ 120%
    )
    .when(
        (floor(datediff(col("maturity_date"), col("open_date")) / 365) > 15) & 
        (col("vof_rate") / col("interest_rate") >= 1.2), 'LN010014'  # CV dài hạn > 180T, HSRR ≥ 120%
    )
    .when(
        floor(datediff(col("maturity_date"), col("open_date")) / 365) < 1, 'LN01005'  # CV ngắn hạn
    )
    .when(
        floor(datediff(col("maturity_date"), col("open_date")) / 365).between(1, 5), 'LN01006'  # CV trung hạn
    )
    .when(
        floor(datediff(col("maturity_date"), col("open_date")) / 365).between(5, 10), 'LN01007'  # CV dài hạn ≤ 120T
    )
    .when(
        floor(datediff(col("maturity_date"), col("open_date")) / 365).between(10, 15), 'LN01008'  # CV dài hạn ≤ 180T
    )
    .when(
        floor(datediff(col("maturity_date"), col("open_date")) / 365) > 15, 'LN01009'  # CV dài hạn > 180T
    )
    
    .when(col("account_type") == 'CV VND LS USD', 'LN01004')
    .when(col("account_type") == 'CV CCSTK', 'LN01003')
    .when(col("account_type") == 'Nợ QH', 'LN01001')
    .when(col("account_type") == 'CV THONG THUONG', 'LN01002')
    .otherwise(None)
)

fact_dp_customer = fact_dp_customer.withColumn(
    "funding_id_loai_hinh",
    when(col("account_type") == 'Business', 'LN02001')
    .when(col("account_type") == 'Individual', 'LN02002')
    .otherwise(None)
)

In [275]:
fact_dp_customer.show()

+----------+----------+------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+-----------------+--------------------+
|process_dt|account_id|account_type|customer_id|customer_type|interest_rate|vof_rate|    balance| open_date|maturity_date|close_date|funding_id_ky_han|funding_id_loai_hinh|
+----------+----------+------------+-----------+-------------+-------------+--------+-----------+----------+-------------+----------+-----------------+--------------------+
|2023-01-01|       100|  Individual|        100|          VIP|         5.21|     7.4| 2393700000|2023-01-01|   2023-04-19|2023-04-19|         LN010010|             LN02002|
|2023-01-01|       555|  Individual|        555|      Regular|         7.73|    9.08| 4623900000|2023-01-01|   2023-12-27|2023-12-28|          LN01005|             LN02002|
|2023-01-01|      1075|  Individual|       1075|      Regular|         4.64|    6.91| 4620100000|2023-01-01|   2023-04-14|2023-04-14|  

### Dựng cấu trúc bảng fact về các tiêu chí báo cáo theo 
### daily : đề xuất tên bảng : fct_funding_daily

In [276]:
from pyspark.sql.types import StructType, StructField, DateType, StringType, FloatType, LongType, BooleanType, TimestampType
from pyspark.sql.functions import col, avg, sum, month, lit
from datetime import datetime
import logging

schema = StructType([
    StructField("process_dt", DateType(), True),          # Cột process_dt kiểu Date
    StructField("funding_id", StringType(), True),        # Cột funding_id kiểu String
    StructField("interest_rate", FloatType(), True),      # Cột interest_rate kiểu Float
    StructField("vof_rate", FloatType(), True),           # Cột vof_rate kiểu Float
    StructField("cof_rate", FloatType(), True),           # Cột cof_rate kiểu Float
    StructField("balance", LongType(), True),             # Cột balance kiểu Long (tương đương int8 trong SQL)
    StructField("avg_balance_month", LongType(), True)    # Cột avg_balance_month kiểu Long (tương đương int8 trong SQL)
])

log_schema = StructType([
    StructField("procedure_name", StringType(), True),   # Tên thủ tục
    StructField("start_time", TimestampType(), True),    # Thời gian bắt đầu
    StructField("end_time", TimestampType(), True),      # Thời gian kết thúc
    StructField("is_successful", BooleanType(), True),   # Trạng thái thành công (True/False)
    StructField("error_log", StringType(), True),        # Thông tin lỗi (nếu có)
    StructField("rec_created_dt", TimestampType(), True) # Thời gian tạo bản ghi
])

log_df = spark_hdfs.createDataFrame(spark_hdfs.sparkContext.emptyRDD(), log_schema)

fct_funding_daily = spark_hdfs.createDataFrame(spark_hdfs.sparkContext.emptyRDD(), schema)

In [277]:
fct_funding_daily.printSchema()

root
 |-- process_dt: date (nullable = true)
 |-- funding_id: string (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- vof_rate: float (nullable = true)
 |-- cof_rate: float (nullable = true)
 |-- balance: long (nullable = true)
 |-- avg_balance_month: long (nullable = true)



In [278]:
log_df.printSchema()

root
 |-- procedure_name: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- is_successful: boolean (nullable = true)
 |-- error_log: string (nullable = true)
 |-- rec_created_dt: timestamp (nullable = true)



In [279]:
log_df.show()

+--------------+----------+--------+-------------+---------+--------------+
|procedure_name|start_time|end_time|is_successful|error_log|rec_created_dt|
+--------------+----------+--------+-------------+---------+--------------+
+--------------+----------+--------+-------------+---------+--------------+



In [280]:
fct_funding_daily.show()

+----------+----------+-------------+--------+--------+-------+-----------------+
|process_dt|funding_id|interest_rate|vof_rate|cof_rate|balance|avg_balance_month|
+----------+----------+-------------+--------+--------+-------+-----------------+
+----------+----------+-------------+--------+--------+-------+-----------------+



### Viết producer về fct_funding_daily

In [305]:
fct_funding_daily = fct_funding_daily.limit(0)
def fct_funding_daily_prc(v_date = None):
    # Thời gian bắt đầu
    start_time = datetime.now()
    is_successful = False
    error_log = None
    process_dt = v_date if v_date else (datetime.now() - timedelta(days=1)).date()

    try:        

        dp_ky_han_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                            .groupBy("funding_id_ky_han") \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                avg("vof_rate").alias("vof_rate"),
                                lit(0).alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumnRenamed("funding_id_ky_han", "funding_id") \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        dp_ky_han_avg_balance_items_df = fact_dp_customer.filter(month("process_dt") == month(lit(process_dt))) \
            .groupBy("funding_id_ky_han") \
            .agg(
                avg("balance").alias("avg_balance_month")
            ).withColumn("process_dt", lit(process_dt)) \
            .withColumnRenamed("funding_id_ky_han", "funding_id") \
            .select("funding_id", "avg_balance_month")
        
        
        
        dp_loai_hinh_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                            .groupBy("funding_id_loai_hinh") \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                avg("vof_rate").alias("vof_rate"),
                                lit(0).alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumnRenamed("funding_id_loai_hinh", "funding_id") \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        dp_loai_hinh_avg_balance_items_df = fact_dp_customer.filter(month("process_dt") == month(lit(process_dt))) \
                    .groupBy("funding_id_loai_hinh") \
                    .agg(
                        avg("balance").alias("avg_balance_month")
                    ).withColumn("process_dt", lit(process_dt)) \
                    .withColumnRenamed("funding_id_loai_hinh", "funding_id") \
                    .select("funding_id", "avg_balance_month")


        dp_final_ky_han_items_df = dp_ky_han_items_df.join(dp_ky_han_avg_balance_items_df, "funding_id", "left")
        dp_final_loai_hinh_items_df = dp_loai_hinh_items_df.join(dp_loai_hinh_avg_balance_items_df, "funding_id", "left")
        dp_final_items_df = dp_final_ky_han_items_df.union(dp_final_loai_hinh_items_df).distinct()

        
        ln_ky_han_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                            .groupBy("funding_id_ky_han") \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                lit(0).alias("vof_rate"),
                                avg("cof_rate").alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumnRenamed("funding_id_ky_han", "funding_id") \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        ln_ky_han_avg_balance_items_df = fact_ln_customer.filter(month("process_dt") == month(lit(process_dt))) \
            .groupBy("funding_id_ky_han") \
            .agg(
                avg("balance").alias("avg_balance_month")
            ).withColumn("process_dt", lit(process_dt)) \
            .withColumnRenamed("funding_id_ky_han", "funding_id") \
            .select("funding_id", "avg_balance_month")


        ln_loai_hinh_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                            .groupBy("funding_id_loai_hinh") \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                lit(0).alias("vof_rate"),
                                avg("cof_rate").alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumnRenamed("funding_id_loai_hinh", "funding_id") \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        ln_loai_hinh_avg_balance_items_df = fact_ln_customer.filter(month("process_dt") == month(lit(process_dt))) \
            .groupBy("funding_id_loai_hinh") \
            .agg(
                avg("balance").alias("avg_balance_month")
            ).withColumn("process_dt", lit(process_dt)) \
            .withColumnRenamed("funding_id_loai_hinh", "funding_id") \
            .select("funding_id", "avg_balance_month")

        ln_final_ky_han_items_df = ln_ky_han_items_df.join(ln_ky_han_avg_balance_items_df, "funding_id", "left")
        ln_final_loai_hinh_items_df = ln_loai_hinh_items_df.join(ln_loai_hinh_avg_balance_items_df, "funding_id", "left")
        ln_final_items_df = ln_final_ky_han_items_df.union(ln_final_loai_hinh_items_df).distinct()


        dp_DP01_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                avg("vof_rate").alias("vof_rate"),
                                lit(0).alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("DP01")) \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        
        dp_DP01_avg_balance_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                           .agg(
                                avg("balance").alias("avg_balance_month")
                            ).withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("DP01")) \
                            .select("funding_id", "avg_balance_month")
        
        dp_DP01_df = dp_DP01_items_df.join(dp_DP01_avg_balance_items_df, "funding_id")

        
        dp_DP02_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                avg("vof_rate").alias("vof_rate"),
                                lit(0).alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("DP02")) \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        dp_DP02_avg_balance_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                           .agg(
                                avg("balance").alias("avg_balance_month")
                            ).withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("DP02")) \
                            .select("funding_id", "avg_balance_month")
        
        dp_DP02_df = dp_DP02_items_df.join(dp_DP02_avg_balance_items_df, "funding_id")
        
        
        dp_DP_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                avg("vof_rate").alias("vof_rate"),
                                lit(0).alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("DP")) \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        dp_DP_avg_balance_items_df = fact_dp_customer.filter(col("process_dt") == process_dt) \
                           .agg(
                                avg("balance").alias("avg_balance_month")
                            ).withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("DP")) \
                            .select("funding_id", "avg_balance_month")
        
        dp_DP_df = dp_DP_items_df.join(dp_DP_avg_balance_items_df, "funding_id")

        
        dp_final_df = dp_final_items_df.union(dp_DP01_df).union(dp_DP02_df).union(dp_DP_df).distinct()


        ln_LN01_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                lit(0).alias("vof_rate"),
                                avg("cof_rate").alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("LN01")) \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        ln_LN01_avg_balance_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                           .agg(
                                avg("balance").alias("avg_balance_month")
                            ).withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("LN01")) \
                            .select("funding_id", "avg_balance_month")
        
        ln_LN01_df = ln_LN01_items_df.join(ln_LN01_avg_balance_items_df, "funding_id")


        ln_LN02_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                lit(0).alias("vof_rate"),
                                avg("cof_rate").alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("LN02")) \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        ln_LN02_avg_balance_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                           .agg(
                                avg("balance").alias("avg_balance_month")
                            ).withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("LN02")) \
                            .select("funding_id", "avg_balance_month")
        
        ln_LN02_df = ln_LN02_items_df.join(ln_LN02_avg_balance_items_df, "funding_id")
        
        ln_LN_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                            .agg(
                                avg("interest_rate").alias("interest_rate"),
                                lit(0).alias("vof_rate"),
                                avg("cof_rate").alias("cof_rate"),
                                sum("balance").alias("balance")
                            )\
                            .withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("LN")) \
                            .select("process_dt", "funding_id", "interest_rate", "vof_rate", "cof_rate", "balance")

        ln_LN_avg_balance_items_df = fact_ln_customer.filter(col("process_dt") == process_dt) \
                           .agg(
                                avg("balance").alias("avg_balance_month")
                            ).withColumn("process_dt", lit(process_dt)) \
                            .withColumn("funding_id", lit("LN")) \
                            .select("funding_id", "avg_balance_month")
        
        ln_LN_df = ln_LN_items_df.join(ln_LN_avg_balance_items_df, "funding_id")


        ln_final_df = ln_final_items_df.union(ln_LN01_df).union(ln_LN02_df).union(ln_LN_df).distinct()


        fct_funding_daily = dp_final_df.union(ln_final_df).distinct()
                
        is_successful = True
    except Exception as e:
        error_log = str(e)
    
    # Thời gian kết thúc
    end_time = datetime.now()
    
    # Ghi log vào bảng log_tracking
    log_data = {
        'procedure_name': 'fct_funding_daily_prc',
        'start_time': start_time,
        'end_time': end_time,
        'is_successful': is_successful,
        'error_log': error_log,
        'rec_created_dt': datetime.now()
    }
    log_data_df = spark_hdfs.createDataFrame([log_data], log_schema)
    
    return fct_funding_daily, log_data_df

    if error_log:
        print(f"Error: {error_log}")
    else:
        print(f"Procedure executed successfully.")

In [306]:
fct_funding_daily, log_df = fct_funding_daily_prc("2023-03-01")

In [307]:
fct_funding_daily.show(100)

+----------+----------+-------------------+------------------+------------------+-------------+--------------------+
|funding_id|process_dt|      interest_rate|          vof_rate|          cof_rate|      balance|   avg_balance_month|
+----------+----------+-------------------+------------------+------------------+-------------+--------------------+
|   LN01006|2023-03-01|  6.434444444444444|              7.25|               0.0|  42934700000| 4.859148051948052E9|
|   LN01005|2023-03-01|  7.017420147420148|7.9799262899262855|               0.0|1419119600000| 3.547766752669677E9|
|  LN010010|2023-03-01|  5.406174496644295| 7.592197986577175|               0.0|1993869700000| 3.268754687062774E9|
|  LN010011|2023-03-01|  5.693636363636364|  7.51090909090909|               0.0|  43072900000| 3.831869326683292E9|
|   LN02002|2023-03-01|  4.839586466165436|   7.1810432330827|               0.0|2138175511618|1.9617743840219078E9|
|   LN02001|2023-03-01|  4.779704641350203|7.0791561181434615|  

In [308]:
log_df.show()

+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|      procedure_name|          start_time|            end_time|is_successful|error_log|      rec_created_dt|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|fct_funding_daily...|2024-11-07 17:48:...|2024-11-07 17:48:...|         true|     NULL|2024-11-07 17:48:...|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+



In [309]:
from pyspark.sql import functions as F

dim_funding_structure.alias("x") \
    .join(fct_funding_daily.alias("y"),
           F.col("x.funding_code") == F.col("y.funding_id"), 
          "left") \
    .select(
        F.col("x.funding_name").alias("NGUỒN VỐN - SỬ DỤNG VỐN"),
        F.coalesce(F.col("y.avg_balance_month"), F.lit(0)).alias("Số dư/dư nợ BQ"),
        F.coalesce(F.col("y.balance"), F.lit(0)).alias("Số dư/dư nợ TĐ"),
        F.coalesce(F.col("y.interest_rate"), F.lit(0)).alias("Lãi suất (%)"),
        F.when(F.col("x.funding_code").like("DP%"), F.coalesce(F.col("y.vof_rate"), F.lit(0)))
         .otherwise(F.coalesce(F.col("y.cof_rate"), F.lit(0))).alias("VOF/COF (%)"),
        (F.when(F.col("x.funding_code").like("DP%"), F.coalesce(F.col("y.vof_rate"), F.lit(0)))
         .otherwise(F.coalesce(F.col("y.cof_rate"), F.lit(0))) - F.coalesce(F.col("y.interest_rate"), F.lit(0)))
         .alias("Margin (%)"),
        F.col("y.process_dt").alias("Ngày báo cáo")
    ) \
    .orderBy(F.col("x.sortorder").asc()).show(100)

+-----------------------+--------------------+--------------+------------------+------------------+-------------------+------------+
|NGUỒN VỐN - SỬ DỤNG VỐN|      Số dư/dư nợ BQ|Số dư/dư nợ TĐ|      Lãi suất (%)|       VOF/COF (%)|         Margin (%)|Ngày báo cáo|
+-----------------------+--------------------+--------------+------------------+------------------+-------------------+------------+
|           A. Nguôn vốn|2.7005301198693314E9| 3513389685950| 4.828677940046152| 7.162482705611075| 2.3338047655649232|  2023-03-01|
|   I/ Tiền gửi TCKT ...|2.7005301198693314E9| 3513389685950| 4.828677940046152| 7.162482705611075| 2.3338047655649232|  2023-03-01|
|                 a. KKH|                 0.0|             0|               0.0|               0.0|                0.0|        NULL|
|                b. 1-3W|                 0.0|             0|               0.0|               0.0|                0.0|        NULL|
|           c. 1-5 tháng|                 0.0|             0|        

## Challenge Bài Test Yody

In [311]:
inventory_data = spark_hdfs.read.csv("hdfs://172.30.2.147:9000/demo_directory/challenge/inventory_data.csv", 
                           header = "true", inferSchema = "true")

In [312]:
rate_sell_data = spark_hdfs.read.csv("hdfs://172.30.2.147:9000/demo_directory/challenge/rate_sell_data.csv", 
                           header = "true", inferSchema = "true")

In [313]:
inventory_data.show()

+---------------+-------------+---------+---------+
|            SKU|group_product|inventory|sell_7day|
+---------------+-------------+---------+---------+
|AKK4001-XDM-4-5|         Kids|      554|        0|
| ALN4036-TRA-XL|    Bassic Nữ|        2|        0|
|  ADN5000-TRA-M|     Phụ Kiện|        4|        1|
|  ADN5000-TRA-L|     Phụ Kiện|        2|        0|
|  APK5001-TRA-4|         Polo|        8|        3|
|  APK5019-TRA-4|         Polo|      159|       26|
| APK5019-VAG-14|         Polo|      201|       11|
| AKM4027-XDM-XL|       Áo Gió|      615|       38|
|  AKN4024-XDM-S|       Áo Gió|      243|       19|
| AKN4024-XDM-XL|       Áo Gió|      101|       15|
|  APK5001-VAG-2|         Polo|        4|        0|
|  APK5028-TRA-4|         Polo|       25|        3|
|  APK5044-TRA-6|         Polo|       74|       18|
| APK5151-TRA-10|         Polo|      239|        9|
|  APK5155-TRA-4|         Polo|       85|        7|
|  APK5155-VAG-2|         Polo|       14|       14|
| APK5167-TR

In [315]:
rate_sell_data.show(truncate=False)

+----------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|store                 |polo               |bassic_nu          |yody_sport         |phu_kien           |kids               |office_nu          |jeans              |bassic_nam         |office_nam         |ao_gio             |ao_phao           |
+----------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|YODY Lê Duẩn 1        |0.0262251660682709 |0.0273985184113117 |0.0169405484367498 |0.0148554976916247 |0.0146757474665954 |0.0215731503860839 |0.0191177302920165 |0.0183377136516788 |0.0202416642090707 |0.0212651319217569 |0.0               |
|YODY Đồng Hới 2       |

Câu 1:	
        - Từ bảng Data, anh/chị cho ra 1 bảng đánh giá thể hiện được 2 yếu tố cần làm rõ:

        + Tồn, Bán của Nhóm Hàng (Phân tích hàng có đủ bán hay không?)
        
        + Tồn, Bán của từng màu trong từng Nhóm Hàng (Phân tích điểm chung giữa các mã trong từng Nhóm Hàng.)

In [337]:
from pyspark.sql.functions import col, substring, when

dim_color = inventory_data.withColumn(
    "name",
    when(substring(col("SKU"), 9, 3) == "TRA", "Trắng")
    .when(substring(col("SKU"), 9, 3) == "XDM", "Xanh đậm")
    .when(substring(col("SKU"), 9, 3) == "VAG", "Vàng")
    .otherwise("Unknown")  # Xử lý trường hợp không khớp
).withColumn("code",
           substring(col("SKU"), 9, 3)
          ).select("code", "name").distinct()

dim_color.show()

+----+--------+
|code|    name|
+----+--------+
| XDM|Xanh đậm|
| VAG|    Vàng|
| TRA|   Trắng|
+----+--------+



In [344]:
dim_type_person = inventory_data.withColumn(
    "name",
    when(substring(col("SKU"), 3, 1) == "N", "Nữ")
    .when(substring(col("SKU"), 3, 1) == "M", "Nam")
    .when(substring(col("SKU"), 3, 1) == "K", "Kid")
    .otherwise("Kid")  # Xử lý trường hợp không khớp
).withColumn("code",
           when(substring(col("SKU"), 3, 1) == "D", "K")
             .otherwise(substring(col("SKU"), 3, 1))
          ).select("code", "name").distinct()

dim_type_person.show()

+----+----+
|code|name|
+----+----+
|   N|  Nữ|
|   K| Kid|
|   M| Nam|
+----+----+



In [354]:
inventory_data.filter(
    ~substring(col("SKU"),3,1).isin("N","M","K")
).show()

+---------------+-------------+---------+---------+
|            SKU|group_product|inventory|sell_7day|
+---------------+-------------+---------+---------+
|  KID3056-TRA-8|         Kids|       13|        0|
|  KID3261-VAG-8|         Kids|       27|        0|
|  KID4034-TRA-6|         Kids|        1|        0|
|  KID3261-TRA-2|         Kids|       12|        0|
| KID3261-TRA-10|         Kids|        7|        0|
|KID4085-TRA-2-3|         Kids|        3|        0|
|  KID4035-VAG-6|         Kids|        1|        0|
| KID3056-TRA-10|         Kids|        3|        0|
|  KID3096-TRA-6|         Kids|        1|        0|
|  KID3261-VAG-2|         Kids|       24|        1|
|  KID3056-TRA-4|         Kids|       15|        0|
|  KID3261-TRA-8|         Kids|       17|        0|
| KID4034-TRA-10|         Kids|        1|        0|
| KID4035-TRA-10|         Kids|        1|        0|
|  KID3056-TRA-6|         Kids|        5|        0|
|  KID3261-TRA-4|         Kids|       43|        1|
| KID4018-TR

In [387]:
## dùng selectExpr để đổi tên khi quá nhiều tên của bản cần đổi tên
from pyspark.sql import functions as F
from pyspark.sql.functions import col, substring, when

fact_sku_daily_eod = inventory_data.alias("x") \
    .join(dim_color.alias("y"), substring(F.col("x.SKU"), 9, 3) == F.col("y.code"), "inner") \
    .join(
        dim_type_person.alias("z"),
        when(substring(col("x.SKU"), 3, 1) == "D", "K")
        .otherwise(substring(col("x.SKU"), 3, 1)) == col("z.code"),
        "inner"
    ) \
    .selectExpr( 
        "x.*", 
        "y.code as id_color", 
        "y.name as code_color", 
        "z.code as id_type_person", 
        "z.name as code_type_person"
    )

fact_sku_daily_eod.show()

+---------------+-------------+---------+---------+--------+----------+--------------+----------------+
|            SKU|group_product|inventory|sell_7day|id_color|code_color|id_type_person|code_type_person|
+---------------+-------------+---------+---------+--------+----------+--------------+----------------+
|AKK4001-XDM-4-5|         Kids|      554|        0|     XDM|  Xanh đậm|             K|             Kid|
| ALN4036-TRA-XL|    Bassic Nữ|        2|        0|     TRA|     Trắng|             N|              Nữ|
|  ADN5000-TRA-M|     Phụ Kiện|        4|        1|     TRA|     Trắng|             N|              Nữ|
|  ADN5000-TRA-L|     Phụ Kiện|        2|        0|     TRA|     Trắng|             N|              Nữ|
|  APK5001-TRA-4|         Polo|        8|        3|     TRA|     Trắng|             K|             Kid|
|  APK5019-TRA-4|         Polo|      159|       26|     TRA|     Trắng|             K|             Kid|
| APK5019-VAG-14|         Polo|      201|       11|     VAG|    

In [422]:
## theo theo từng mã sản phẩm
fact_sku_daily_eod \
    .groupBy("SKU") \
    .agg(
        sum("inventory").alias("total_inventory"),
        sum("sell_7day").alias("total_sell_7day")
    ) \
    .withColumn("is_enough_in_7day",
        when((col("total_sell_7day") <= col("total_inventory")), True) \
        .otherwise(False)
    ) \
    .withColumnRenamed("SKU", "id_product") \
    .show()

+---------------+---------------+---------------+-----------------+
|     id_product|total_inventory|total_sell_7day|is_enough_in_7day|
+---------------+---------------+---------------+-----------------+
|  APK5179-VAG-4|             10|             15|            false|
|  APM3639-TRA-L|             91|             13|             true|
|  AKM4027-XDM-L|            586|             64|             true|
|APM3733-TRA-2XL|              1|              0|             true|
|  TSK5011-TRA-6|            100|              7|             true|
| SJN3070-XDM-27|            194|             12|             true|
| SJN4010-TRA-25|             71|             50|             true|
| TSK5117-VAG-12|             26|              3|             true|
|  TSK5097-VAG-8|             53|             17|             true|
| VEM4007-XDM-XL|             83|              1|             true|
|  APK5129-VAG-4|             48|              5|             true|
| APM4375-TRA-XL|              1|              9

In [421]:
## theo từng sản phẩm
fact_sku_daily_eod \
    .groupBy("group_product") \
    .agg(
        sum("inventory").alias("total_inventory"),
        sum("sell_7day").alias("total_sell_7day")
    ) \
    .withColumn("is_enough_in_7day",
        when((col("total_sell_7day") <= col("total_inventory")), True) \
        .otherwise(False)
    ) \
    .show()

+-------------+---------------+---------------+-----------------+
|group_product|total_inventory|total_sell_7day|is_enough_in_7day|
+-------------+---------------+---------------+-----------------+
|       Áo Gió|           2848|            297|             true|
|         Polo|         197446|          22279|             true|
|   YODY Sport|          15385|           4613|             true|
|   Bassic Nam|          19828|           2804|             true|
|      Áo Phao|            250|              1|             true|
|         Kids|          34261|           4082|             true|
|   Office Nam|          11165|           2782|             true|
|    Bassic Nữ|          30664|           5704|             true|
|     Phụ Kiện|           3898|           1464|             true|
|    Office Nữ|           5393|           1393|             true|
|        Jeans|          16373|           4094|             true|
+-------------+---------------+---------------+-----------------+



In [420]:
## theo từng màu
fact_sku_daily_eod \
    .groupBy("id_color", "code_color") \
    .agg(
        sum("inventory").alias("total_inventory"),
        sum("sell_7day").alias("total_sell_7day")
    ) \
    .withColumn("is_enough_in_7day",
        when((col("total_sell_7day") <= col("total_inventory")), True) \
        .otherwise(False)
    ) \
    .show()

+--------+----------+---------------+---------------+-----------------+
|id_color|code_color|total_inventory|total_sell_7day|is_enough_in_7day|
+--------+----------+---------------+---------------+-----------------+
|     XDM|  Xanh đậm|          34433|           6176|             true|
|     VAG|      Vàng|         141674|          12800|             true|
|     TRA|     Trắng|         161404|          30537|             true|
+--------+----------+---------------+---------------+-----------------+



In [419]:
## theo từng loại người
fact_sku_daily_eod \
    .groupBy("id_type_person", "code_type_person") \
    .agg(
        sum("inventory").alias("total_inventory"),
        sum("sell_7day").alias("total_sell_7day")
    ) \
    .withColumn("is_enough_in_7day",
        when((col("total_sell_7day") <= col("total_inventory")), True) \
        .otherwise(False)
    ) \
    .show()

+--------------+----------------+---------------+---------------+-----------------+
|id_type_person|code_type_person|total_inventory|total_sell_7day|is_enough_in_7day|
+--------------+----------------+---------------+---------------+-----------------+
|             N|              Nữ|         137224|          21736|             true|
|             K|             Kid|          76571|           7026|             true|
|             M|             Nam|         123716|          20751|             true|
+--------------+----------------+---------------+---------------+-----------------+



In [418]:
## theo từng nhóm sản phẩm, màu, loại người
fact_sku_daily_eod \
    .groupBy("group_product", "code_color", "code_type_person") \
    .agg(
        sum("inventory").alias("total_inventory"),
        sum("sell_7day").alias("total_sell_7day")
    ) \
    .withColumn("is_enough_in_7day",
        when((col("total_sell_7day") <= col("total_inventory")), True) \
        .otherwise(False)
    ) \
    .show()

+-------------+----------+----------------+---------------+---------------+-----------------+
|group_product|code_color|code_type_person|total_inventory|total_sell_7day|is_enough_in_7day|
+-------------+----------+----------------+---------------+---------------+-----------------+
|   YODY Sport|     Trắng|              Nữ|           2452|           1180|             true|
|   YODY Sport|  Xanh đậm|             Nam|           3833|            797|             true|
|         Polo|     Trắng|             Kid|          22119|           1487|             true|
|    Bassic Nữ|      Vàng|              Nữ|          10134|           1113|             true|
|   Office Nam|     Trắng|             Nam|           4342|           1552|             true|
|   YODY Sport|     Trắng|             Kid|            196|             25|             true|
|    Office Nữ|     Trắng|              Nữ|           4401|           1130|             true|
|     Phụ Kiện|      Vàng|             Kid|            100| 

Câu 2:	- Đánh giá hàng hóa từ Bảng Data để đưa ra số lượng đặt hàng (hoặc cần luân chuyển về) tới từng mã chi tiết. Đảm bảo hàng hóa đủ bán trong vòng 60 ngày (hoặc tùy theo đánh giá về số ngày bán của Người làm bài tập).

* Thông tin thêm: Hàng về sẽ trong vòng từ 20 đến 45 ngày tùy vào chất liệu, mẫu mã, số lượng đặt.

In [423]:
## theo theo từng mã sản phẩm
fact_sku_daily_eod \
    .groupBy("SKU","group_product") \
    .agg(
        sum("inventory").alias("total_inventory"),
        sum("sell_7day").alias("total_sell_7day")
    ) \
    .withColumn("is_enough_in_7day",
        when((col("total_sell_7day") <= col("total_inventory")), True) \
        .otherwise(False)
    ) \
    .withColumnRenamed("SKU", "id_product") \
    .show()

+---------------+-------------+---------------+---------------+-----------------+
|     id_product|group_product|total_inventory|total_sell_7day|is_enough_in_7day|
+---------------+-------------+---------------+---------------+-----------------+
|  APM3543-VAG-M|         Polo|              1|              0|             true|
|APM5243-VAG-4XL|         Polo|             33|              2|             true|
|  TSN5180-TRA-S|    Bassic Nữ|             71|             13|             true|
|  TSN5328-TRA-L|    Bassic Nữ|             80|              6|             true|
|  TSK5127-TRA-8|         Kids|             42|              5|             true|
|  TSK5029-TRA-6|         Kids|             86|             21|             true|
|  APN3422-TRA-S|         Polo|              1|              0|             true|
|APM5377-VAG-3XL|         Polo|            127|              1|             true|
|  APM5065-TRA-L|         Polo|             67|             69|            false|
|  QSK5039-VAG-8

In [391]:
from pyspark.sql import functions as F

dim_store = rate_sell_data\
    .withColumn("rank", F.row_number().over(Window.orderBy("store"))) \
    .withColumn("code", F.concat(
        lit("S"),
        F.lpad(F.col("rank").cast("string"), 4, "0")
    )) \
    .select("code", "store")

dim_store.show()

+-----+------------------+
| code|             store|
+-----+------------------+
|S0001|       YODY An Khê|
|S0002|      YODY An Nhơn|
|S0003|      YODY BUÔN HỒ|
|S0004|    YODY BUÔN TRẤP|
|S0005|       YODY Ba Đồn|
|S0006|YODY Buôn Ma Thuột|
|S0007|     YODY Bình Sơn|
|S0008|     YODY Bồng Sơn|
|S0009|       YODY Chư Sê|
|S0010|    YODY Cẩm Xuyên|
|S0011|       YODY Ea Kar|
|S0012|      YODY Gia Lai|
|S0013|    YODY Gia Nghĩa|
|S0014|     YODY Hoàn Lão|
|S0015|       YODY Hà Lam|
|S0016|      YODY Hà Tĩnh|
|S0017|    YODY Hương Khê|
|S0018|        YODY Hồ Xá|
|S0019|    YODY Hồng Lĩnh|
|S0020|       YODY Hội An|
+-----+------------------+
only showing top 20 rows



In [15]:
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder \
    .appName("CV compare") \
    .master("local[2]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.network.timeout", "12000s") \
    .config("spark.executor.heartbeatInterval", "2000s") \
    .getOrCreate()


In [19]:
from pyspark.sql import SparkSession

# Tạo Spark session
spark = SparkSession.builder \
    .appName("Read from HTTP") \
    .getOrCreate()

# Cấu hình URL HTTP để Spark có thể đọc
spark.conf.set("spark.hadoop.fs.http.impl", "org.apache.hadoop.fs.http.HttpFileSystem")

# Đọc dữ liệu từ HTTP như file hệ thống
url = "http://172.30.2.147:8000/citizens_data.csv"
citizens_rdd_sv = spark.sparkContext.textFile(url)

# Xử lý dữ liệu
citizens_rdd_sv.collect()


IllegalArgumentException: Invalid start or len parameter

In [14]:
start_time_sv_csv = time.time()
citizens_rdd_sv = spark.sparkContext.textFile("http://172.30.2.147:8000/citizens_data.csv", minPartitions=2)
header = citizens_rdd_sv.first()
data_citizens_rdd_sv = citizens_rdd_sv.filter(lambda line: line != header)

cities_in_country = data_citizens_rdd_sv \
        .map(lambda x: (x.split(",")[4],1)) \
        .reduceByKey(lambda x,y: x+y)
print(cities_in_country.take(5))

end_time_sv_csv = time.time()
time_sv_csv = end_time_sv_csv - start_time_sv_csv
print("Thời gian chạy với Server: {:.2f} giây".format(time_sv_csv))


IllegalArgumentException: Invalid start or len parameter